In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
import json
with open("service1906_1506.json") as of:
    data = json.load(of)

In [ ]:
computes = [c for c in data.keys() if c!="timespan"]
variables = [v for v in data[computes[0]] if v!='index' and v!='arrJob_scheduling']

In [ ]:
#Check empty array
def getEmptyArr(data, c):
    cObj = data[c]
    cDf = pd.DataFrame()
    cDf['compute'] = [c for _ in data['timespan']]
    cDf['timespan'] = data['timespan']
    for v in variables:
        vArr = np.array(cObj[v])
        if len(vArr)==0:
            print('c=', c)
            print('v=', v)
for c in computes:
    getEmptyArr(data, c)

In [ ]:
def addTarget(cDf, predictedVar, predictedStep, target):
    cDf[target] = cDf[predictedVar].shift(-predictedStep)
    cDf.dropna(inplace=True)
    return cDf

In [ ]:
def getComputeDf(data, c, predictedVar, predictedStep, target):
    cObj = data[c]
    cDf = pd.DataFrame()
    cDf['compute'] = [c for _ in data['timespan']]
    cDf['timespan'] = data['timespan']
    for v in variables:
        vArr = np.array(cObj[v])
        if len(vArr)==0:
            return None
        else:
            for i in range(len(vArr[0])):
                cDf[v+str(i)] = vArr[:, i]
    cDf['timespan'] = pd.to_datetime(cDf['timespan'])
    return addTarget(cDf, predictedVar, predictedStep, target)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import FileLink
import codecs, json

In [ ]:
def exportNPArrayToJson(a, fileName):
    b = a.tolist()
    json.dump(b, codecs.open(fileName, 'w', encoding='utf-8'))

In [ ]:
def splitTrainTest(predictedVar):
    target = predictedVar + "_target"
    predictedSteps = 4
    df = pd.concat([x for x in [getComputeDf(data, c, predictedVar, predictedSteps, target) for c in computes] if type(x)!="NoneType"])
    
    df = df.reset_index().drop('index', axis=1)
    features = [x for x in df.columns if x not in ['compute', 'timespan', target]]
        
    X_dfs = []
    y = []
    numberOfSequences = 1
    # generate training data.
    for compute in computes:
        cDf = df[df['compute']==compute]
        if(len(cDf) > sequenceSteps):
            randSteps = np.random.randint(0, len(cDf)-sequenceSteps, numberOfSequences)
            for randStep in randSteps:
                X_dfs.append(cDf.iloc[randStep:randStep+sequenceSteps])
                y.append(X_dfs[-1][target].values[-1])

    X_train_dfs, X_test_dfs, y_train, y_test = train_test_split(X_dfs, y, test_size=0.33)
            
    #Scale data

    # combine the training data to create a scaler
    train_dfs = pd.concat(X_train_dfs)

    scaler = StandardScaler().fit(train_dfs[features].values)
    
    X_train = np.array([scaler.transform(item[features].values) for item in X_train_dfs])
    X_test = np.array([scaler.transform(item[features].values) for item in X_test_dfs])
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    exportNPArrayToJson(X_train, 'newData/'+target+'_X_train_HPCC_'+str(sequenceSteps)+'.json')
    exportNPArrayToJson(X_test, 'newData/'+target+'_X_test_HPCC_'+str(sequenceSteps)+'.json')
    exportNPArrayToJson(y_train, 'newData/'+target+'_y_train_HPCC_'+str(sequenceSteps)+'.json')
    exportNPArrayToJson(y_test, 'newData/'+target+'_y_test_HPCC_'+str(sequenceSteps)+'.json')
    
    FileLink('newData/'+target+'_X_train_HPCC_'+str(sequenceSteps)+'.json')
    FileLink('newData/'+target+'_X_test_HPCC_'+str(sequenceSteps)+'.json')
    FileLink('newData/'+target+'_y_train_HPCC_'+str(sequenceSteps)+'.json')
    FileLink('newData/'+target+'_y_test_HPCC_'+str(sequenceSteps)+'.json')
    
    return (X_train, X_test, y_train, y_test)

In [10]:
from keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
import tensorflow as tf

SEED = 1711
import os
import random as rn
import numpy as np
from tensorflow import set_random_seed

os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
set_random_seed(SEED)
rn.seed(SEED)

from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)
# from keras import backend as K
# K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=36, inter_op_parallelism_threads=36)))


def createModel(l1Nodes, l2Nodes, d1Nodes, d2Nodes, inputShape):
    # input layer
    lstm1 = LSTM(l1Nodes, input_shape=inputShape, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    do1 = Dropout(0.2)
    
    lstm2 = LSTM(l2Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    do2 = Dropout(0.2)
    
    flatten = Flatten()
    
    dense1 = Dense(d1Nodes, activation='relu')
    do3 = Dropout(0.2)
    
    dense2 = Dense(d2Nodes, activation='relu')
    do4 = Dropout(0.2)
    
    # output layer
    outL = Dense(1, activation='relu')
    # combine the layers
#     layers = [lstm1, do1, lstm2, do2, dense1, do3, dense2, do4, outL]
#     layers = [lstm1, lstm2, flatten,  dense1, dense2, outL]
#     layers = [lstm1, flatten,  dense1, outL]
    layers = [lstm1, lstm2, flatten,  dense1, outL]

    # create the model
    model = Sequential(layers)
    model.compile(optimizer='adam', loss='mse')
    return model

Using TensorFlow backend.


In [11]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
# ten fold
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True)
from datetime import datetime

sequenceSteps = 100

msescores = []
model_config = "config_8_4_2_ts"+str(sequenceSteps)

tempAllVariables = ['arrTemperature0', 'arrTemperature1', 'arrTemperature2', 'arrCPU_load0', 'arrMemory_usage0', 'arrFans_health0', 'arrFans_health1', 'arrFans_health2', 'arrFans_health3', 'arrPower_usage0']
allVariables = ['arrTemperature0']
timeList = []



for feature in allVariables:
    print(feature)
    X_train, X_test, y_train, Y_test = splitTrainTest(feature)

    for training_time in range(0,3):
        start_time = datetime.now()
        print(start_time)

        counter= 0
        for trainIdx, testIdx in kfold.split(X_train, y_train):
            # create callbacks
            model_path = feature+'_training_time_'+str(training_time)+'_best_model_fold_'+str(counter)+"_"+model_config+'.h5'
            mc = ModelCheckpoint('newModel/'+model_path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)
            es = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1)
            # create model
            model = createModel(l1Nodes = 8, l2Nodes = 4, d1Nodes = 2, d2Nodes = 0, inputShape= (X_train.shape[1], X_train.shape[2]))
            model.fit(X_train[trainIdx], y_train[trainIdx], validation_data=(X_train[testIdx], y_train[testIdx]), batch_size=32, epochs=40, callbacks=[mc, es])
            # Done load the best model of this fold
            saved_model = load_model('newModel/' + model_path)
            msescores.append({'path': 'newModel/' + model_path, 'mse': saved_model.evaluate(X_train[testIdx], y_train[testIdx])})
            counter = counter + 1
        
#             tfjs.converters.save_keras_model(saved_model, 'newModel/'+model_path)

        end_time = datetime.now()
    
        timeList.append({'start': start_time, 'end': end_time, 'time': datetime.timestamp(end_time)-datetime.timestamp(start_time)})

arrTemperature0
2019-11-06 16:35:53.888283
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 249 samples, validate on 63 samples
Epoch 1/40
249/249 [==============================] - 1s 4ms/step - loss: 3528.6710 - val_loss: 3231.7858

Epoch 00001: val_loss improved from inf to 3231.78578, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_4_2_ts100.h5
Epoch 2/40
249/249 [==============================] - 0s 1ms/step - loss: 3489.2711 - val_loss: 3197.9743

Epoch 00002: val_loss improved from 3231.78578 to 3197.97429, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_4_2_ts100.h5
Epoch 3/40
249/249 [==============================] - 0s 1ms/step - loss: 3440.8999 - val_loss: 3146.7459

Epoch 00003: val_loss improved from 3197.97429 to 3146.74588, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_4_2_ts100.h5
Epoch 4/40
249/249 [===================

249/249 [==============================] - 0s 1ms/step - loss: 147.6053 - val_loss: 149.7085

Epoch 00029: val_loss improved from 150.12022 to 149.70851, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_4_2_ts100.h5
Epoch 30/40
249/249 [==============================] - 0s 1ms/step - loss: 140.3759 - val_loss: 148.9658

Epoch 00030: val_loss improved from 149.70851 to 148.96576, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_4_2_ts100.h5
Epoch 31/40
249/249 [==============================] - 0s 1ms/step - loss: 134.6197 - val_loss: 146.7879

Epoch 00031: val_loss improved from 148.96576 to 146.78795, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_4_2_ts100.h5
Epoch 32/40
249/249 [==============================] - 0s 1ms/step - loss: 130.4909 - val_loss: 144.8781

Epoch 00032: val_loss improved from 146.78795 to 144.87808, saving model to newModel/arrTemperature0_training_time_0_

249/249 [==============================] - 0s 1ms/step - loss: 1036.3395 - val_loss: 1125.1999

Epoch 00020: val_loss improved from 1303.39105 to 1125.19994, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_1_config_8_4_2_ts100.h5
Epoch 21/40
249/249 [==============================] - 0s 1ms/step - loss: 889.6964 - val_loss: 969.7596

Epoch 00021: val_loss improved from 1125.19994 to 969.75963, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_1_config_8_4_2_ts100.h5
Epoch 22/40
249/249 [==============================] - 0s 1ms/step - loss: 763.8671 - val_loss: 833.8454

Epoch 00022: val_loss improved from 969.75963 to 833.84537, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_1_config_8_4_2_ts100.h5
Epoch 23/40
249/249 [==============================] - 0s 1ms/step - loss: 659.9921 - val_loss: 715.6829

Epoch 00023: val_loss improved from 833.84537 to 715.68295, saving model to newModel/arrTemperature0_training_ti

250/250 [==============================] - 0s 1ms/step - loss: 339.6527 - val_loss: 298.6821

Epoch 00011: val_loss improved from 417.18520 to 298.68214, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_2_config_8_4_2_ts100.h5
Epoch 12/40
250/250 [==============================] - 0s 1ms/step - loss: 242.8030 - val_loss: 246.5212

Epoch 00012: val_loss improved from 298.68214 to 246.52122, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_2_config_8_4_2_ts100.h5
Epoch 13/40
250/250 [==============================] - 0s 1ms/step - loss: 199.0742 - val_loss: 220.8498

Epoch 00013: val_loss improved from 246.52122 to 220.84980, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_2_config_8_4_2_ts100.h5
Epoch 14/40
250/250 [==============================] - 0s 1ms/step - loss: 170.2922 - val_loss: 194.1321

Epoch 00014: val_loss improved from 220.84980 to 194.13212, saving model to newModel/arrTemperature0_training_time_0_

250/250 [==============================] - 0s 1ms/step - loss: 311.0707 - val_loss: 251.6569

Epoch 00020: val_loss improved from 311.89215 to 251.65693, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_3_config_8_4_2_ts100.h5
Epoch 21/40
250/250 [==============================] - 0s 1ms/step - loss: 262.3326 - val_loss: 213.1806

Epoch 00021: val_loss improved from 251.65693 to 213.18061, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_3_config_8_4_2_ts100.h5
Epoch 22/40
250/250 [==============================] - 0s 1ms/step - loss: 231.5287 - val_loss: 188.2110

Epoch 00022: val_loss improved from 213.18061 to 188.21102, saving model to newModel/arrTemperature0_training_time_0_best_model_fold_3_config_8_4_2_ts100.h5
Epoch 23/40
250/250 [==============================] - 0s 1ms/step - loss: 209.3479 - val_loss: 169.9320

Epoch 00023: val_loss improved from 188.21102 to 169.93201, saving model to newModel/arrTemperature0_training_time_0_

...

249/249 [==============================] - 0s 1ms/step - loss: 2166.0291 - val_loss: 1730.9795

Epoch 00009: val_loss improved from 2060.78115 to 1730.97955, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_1_config_8_4_2_ts100.h5
Epoch 10/40
249/249 [==============================] - 0s 1ms/step - loss: 1735.4813 - val_loss: 1387.8619

Epoch 00010: val_loss improved from 1730.97955 to 1387.86187, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_1_config_8_4_2_ts100.h5
Epoch 11/40
249/249 [==============================] - 0s 1ms/step - loss: 1316.6326 - val_loss: 1055.3182

Epoch 00011: val_loss improved from 1387.86187 to 1055.31817, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_1_config_8_4_2_ts100.h5
Epoch 12/40
249/249 [==============================] - 0s 1ms/step - loss: 930.3872 - val_loss: 774.1683

Epoch 00012: val_loss improved from 1055.31817 to 774.16831, saving model to newModel/arrTemperature0_tra

63/63 [==============================] - 2s 30ms/step
Train on 250 samples, validate on 62 samples
Epoch 1/40
250/250 [==============================] - 4s 14ms/step - loss: 3462.8722 - val_loss: 3660.2272

Epoch 00001: val_loss improved from inf to 3660.22718, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_2_config_8_4_2_ts100.h5
Epoch 2/40
250/250 [==============================] - 0s 1ms/step - loss: 3462.7281 - val_loss: 3660.0824

Epoch 00002: val_loss improved from 3660.22718 to 3660.08240, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_2_config_8_4_2_ts100.h5
Epoch 3/40
250/250 [==============================] - 0s 1ms/step - loss: 3462.5852 - val_loss: 3659.9428

Epoch 00003: val_loss improved from 3660.08240 to 3659.94275, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_2_config_8_4_2_ts100.h5
Epoch 4/40
250/250 [==============================] - 0s 2ms/step - loss: 3462.4486 - val_loss: 3659.8099

Ep


Epoch 00031: val_loss improved from 3658.05970 to 3658.03186, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_2_config_8_4_2_ts100.h5
Epoch 32/40
250/250 [==============================] - 0s 2ms/step - loss: 3460.5834 - val_loss: 3658.0053

Epoch 00032: val_loss improved from 3658.03186 to 3658.00525, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_2_config_8_4_2_ts100.h5
Epoch 33/40
250/250 [==============================] - 0s 1ms/step - loss: 3460.5576 - val_loss: 3657.9804

Epoch 00033: val_loss improved from 3658.00525 to 3657.98035, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_2_config_8_4_2_ts100.h5
Epoch 34/40
250/250 [==============================] - 0s 1ms/step - loss: 3460.5334 - val_loss: 3657.9569

Epoch 00034: val_loss improved from 3657.98035 to 3657.95691, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_2_config_8_4_2_ts100.h5
Epoch 35/40
250/250 [==================

250/250 [==============================] - 0s 1ms/step - loss: 220.7699 - val_loss: 176.9850

Epoch 00022: val_loss improved from 188.89258 to 176.98497, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_3_config_8_4_2_ts100.h5
Epoch 23/40
250/250 [==============================] - 0s 1ms/step - loss: 208.0646 - val_loss: 187.7861

Epoch 00023: val_loss did not improve from 176.98497
Epoch 24/40
250/250 [==============================] - 0s 1ms/step - loss: 195.0423 - val_loss: 238.4029

Epoch 00024: val_loss did not improve from 176.98497
Epoch 25/40
250/250 [==============================] - 0s 1ms/step - loss: 197.0584 - val_loss: 185.8481

Epoch 00025: val_loss did not improve from 176.98497
Epoch 00025: early stopping
62/62 [==============================] - 2s 36ms/step
Train on 250 samples, validate on 62 samples
Epoch 1/40
250/250 [==============================] - 4s 16ms/step - loss: 3376.4011 - val_loss: 3933.9081

Epoch 00001: val_loss improved from i

250/250 [==============================] - 0s 2ms/step - loss: 104.0861 - val_loss: 135.9298

Epoch 00029: val_loss improved from 140.17937 to 135.92981, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_4_config_8_4_2_ts100.h5
Epoch 30/40
250/250 [==============================] - 0s 1ms/step - loss: 101.7401 - val_loss: 130.2117

Epoch 00030: val_loss improved from 135.92981 to 130.21171, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_4_config_8_4_2_ts100.h5
Epoch 31/40
250/250 [==============================] - 0s 2ms/step - loss: 98.3223 - val_loss: 127.1471

Epoch 00031: val_loss improved from 130.21171 to 127.14714, saving model to newModel/arrTemperature0_training_time_2_best_model_fold_4_config_8_4_2_ts100.h5
Epoch 32/40
250/250 [==============================] - 0s 2ms/step - loss: 95.7694 - val_loss: 126.2280

Epoch 00032: val_loss improved from 127.14714 to 126.22801, saving model to newModel/arrTemperature0_training_time_2_be

In [12]:
print ('start time: ',  datetime.timestamp(start_time))
print ('end time:', datetime.timestamp(end_time))
print ('training time: ', datetime.timestamp(end_time)-datetime.timestamp(start_time))

start time:  1573080036.861314
end time: 1573080264.857186
training time:  227.99587202072144


In [13]:
msescores

[{'path': 'newModel/arrTemperature0_training_time_0_best_model_fold_0_config_8_4_2_ts100.h5',
  'mse': 121.60293724423363},
 {'path': 'newModel/arrTemperature0_training_time_0_best_model_fold_1_config_8_4_2_ts100.h5',
  'mse': 173.6720670669798},
 {'path': 'newModel/arrTemperature0_training_time_0_best_model_fold_2_config_8_4_2_ts100.h5',
  'mse': 118.82467479090536},
 {'path': 'newModel/arrTemperature0_training_time_0_best_model_fold_3_config_8_4_2_ts100.h5',
  'mse': 83.63773370558216},
 {'path': 'newModel/arrTemperature0_training_time_0_best_model_fold_4_config_8_4_2_ts100.h5',
  'mse': 144.50047253024192},
 {'path': 'newModel/arrTemperature0_training_time_1_best_model_fold_0_config_8_4_2_ts100.h5',
  'mse': 136.7141849578373},
 {'path': 'newModel/arrTemperature0_training_time_1_best_model_fold_1_config_8_4_2_ts100.h5',
  'mse': 125.5921652657645},
 {'path': 'newModel/arrTemperature0_training_time_1_best_model_fold_2_config_8_4_2_ts100.h5',
  'mse': 194.05529735934348},
 {'path': 'n

In [14]:
# timeList
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_29 (LSTM)               (None, 100, 8)            608       
_________________________________________________________________
lstm_30 (LSTM)               (None, 100, 4)            208       
_________________________________________________________________
flatten_15 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 2)                 802       
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 3         
Total params: 1,621
Trainable params: 1,621
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Plot and see data distribution

from matplotlib import pyplot as plt
import seaborn as sns

def plotAttrDataOfId(data, compute, features):
    plt.figure(figsize=(30, 20))
    for i, v in enumerate(features):
        plt.subplot(10, 3, i+1)
        cDf = df[df['compute']==compute]
        plt.plot(cDf['timespan'], cDf[v])
        plt.title(v)
        plt.tight_layout()

In [ ]:
# for x in np.random.randint(0, len(computes), 3):
#     plotAttrDataOfId(df, computes[x], features)

In [ ]:
def plotDataDistribution(data, features):
    plt.figure(figsize=(30, 10))
    for i, v in enumerate(features):
        plt.subplot(3, 10, i+1)
        sns.distplot(list(data[v].values))
        plt.title(v)
    plt.tight_layout()

In [ ]:
# plotDataDistribution(df, features)

In [ ]:
from datetime import datetime
print(datetime.now().strftime("%H_%M_%S"))

In [ ]:
[i for i in range(1,11
                 )]

In [ ]:
sns.distplot(y_train)

In [ ]:
sns.distplot(y_test)

In [ ]:
print (type(X_dfs[0]))